In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.set_default_tensor_type('torch.DoubleTensor')
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
import os 
import utils
import mlmodel

from datetime import datetime
import re

/home/oyoungquist/.conda/envs/rina/lib/python3.11/site-packages/torch/__init__.py:747: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /opt/conda/conda-bld/pytorch_1712608853085/work/torch/csrc/tensor/python_tensor.cpp:431.)
  _C._set_default_tensor_type(t)


In [2]:
dim_a = 3
features = ['q', 'q_dot', 'tau_cmd']
label = 'tau_residual_cmd'

labels = ["FR_hip", "FR_knee", "FR_foot", "FL_hip", "FL_knee", "FL_foot",
          "RR_hip", "RR_knee", "RR_foot", "RL_hip", "RL_knee", "RL_foot"]

# Training data collected from the neural-fly drone
dataset = 'rina' 
# dataset_folder = '/home/hcr/Research/DARoSLab/DARoS-Core/lcm_converted_log/04_01_2024_eval/unitree_estimate/eval_data/'

dataset_folder = "/home/oyoungquist/Research/RINA/rina/data/lcm_converted_log/05_17_2024_formal/eval_data/"


cwd = os.getcwd()

output_path_base = os.path.join(cwd, "training_results", "05_16_202413_27_47_m")

modelname = f"{dataset}_dim-a-{dim_a}_{'-'.join(features)}" # 'intel-aero_fa-num-Tsp_v-q-pwm'

print(modelname)

rina_dim-a-3_q-q_dot-tau_cmd


In [3]:
RawData = utils.load_data(dataset_folder)
TestData = utils.format_data(RawData, features=features, output=label)

In [4]:
# test_data_images = os.path.join(output_path_base, "eval_condition_data_images")

# if not os.path.exists(test_data_images):
#     os.makedirs(test_data_images)

# for data in TestData:
#     print(data.meta['condition'])
#     utils.plot_subdataset(data, features, labels, os.path.join(test_data_images, "{:s}.png".format(data.meta['condition'])), title_prefix="(Testing Data)")

In [5]:
options = {}
options['dim_x'] = TestData[0].X.shape[1]
options['dim_y'] = TestData[0].Y.shape[1]
options['num_c'] = len(TestData)
print('dims of (x, y) are', (options['dim_x'], options['dim_y']))
print('there are ' + str(options['num_c']) + ' different conditions')

# Set hyperparameters
options['features'] = features
options['dim_a'] = dim_a
options['loss_type'] = 'crossentropy-loss'

options['shuffle'] = True # True: shuffle trajectories to data points
options['K_shot'] = 32 # number of K-shot for least square on a
options['phi_shot'] = 256 # batch size for training phi

options['alpha'] = 0.01 # adversarial regularization loss2
options['learning_rate'] = 5e-4
options['frequency_h'] = 2 # how many times phi is updated between h updates, on average
options['SN'] = 2. # maximum single layer spectral norm of phi
options['gamma'] = 10. # max 2-norm of a
options['num_epochs'] = 2000


options['output_path'] = output_path_base

dims of (x, y) are (36, 6)
there are 4 different conditions


In [6]:
modelname = "rina_dim-a-3_q-q_dot-tau_cmd"
model_path = os.path.join("/home/oyoungquist/Research/RINA/rina/training_results/05_23_202412_04_09_cmd_residual/models", (modelname + '-epoch-' + str(1500)))
final_model = mlmodel.load_model(modelname = model_path)

In [7]:
# phi_net = final_model.phi
# h_net = final_model.h

# eval_adapt_start = 0
# eval_adapt_end = 2500
# eval_val_start = 2500
# eval_val_end = 5000

# # vis_output_path_prefix_training_data = os.path.join(options["output_path"], "eval_iamges", "training")
# # vis_output_path_prefix_testing_data = os.path.join(options["output_path"], "eval_iamges", "testing")

# vis_output_path_prefix_training_data = os.path.join("/home/hcr/Research/DARoSLab/rina/training_results/05_16_202413_27_47_m/eval_condition_images/training")
# vis_output_path_prefix_testing_data = os.path.join("/home/hcr/Research/DARoSLab/rina/training_results/05_16_202413_27_47_m/eval_condition_images/testing")

In [8]:
# for i, data in enumerate(TestData):
#     print('------------------------------')
#     print(data.meta['condition'] + ':')
#     print(len(data.X))
#     file_name = "{:s}.png".format(data.meta['condition'])
#     mlmodel.vis_validation(t=data.meta['steps'], x=data.X, y=data.Y, phi_net=phi_net, h_net=h_net, 
#                            idx_adapt_start=eval_adapt_start, idx_adapt_end=eval_adapt_end, 
#                            idx_val_start=eval_val_start, idx_val_end=eval_val_end, c=TestData[i].C, options=options,
#                            output_path_prefix=vis_output_path_prefix_testing_data, output_name=file_name)

In [9]:
# for data in TestData:
#     image_name = "{:s}_errors_hist.png".format(data.meta['condition'])
#     error_1, error_2, error_3 = mlmodel.error_statistics_hist(data.X, data.Y, phi_net, h_net, options, vis_output_path_prefix_testing_data, image_name)
#     print('**** :', data.meta['condition'], '****')
#     print(f'Before learning: MSE is {error_1: .2f}')
#     print(f'Mean predictor: MSE is {error_2: .2f}')
#     print(f'After learning phi(x): MSE is {error_3: .2f}')
#     print('')

In [10]:
# convert the trained python model to a Torch.Script model
# An example input you would normally provide to your model's forward() method.
example = torch.rand(1, 36)

# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_module = torch.jit.trace(final_model.phi, example)

# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_module = torch.jit.trace(final_model.phi, example)

# testing the traced output
print(traced_script_module(example))

# save-out the scripted model
traced_script_module.save("traced_rina_model_cmd_error.pt")

tensor([[ 0.5392, -0.3284,  1.0000]], grad_fn=<CatBackward0>)
